In [3]:
'''
Load data to a database
'''

# from hmmlearn import hmm
import numpy as np
import pandas as pd
from collections import defaultdict
import datetime
# import pykovy-master/ 
from pykovy.src.pykovy import chain
import sqlite3
from sqlalchemy import create_engine

In [4]:
df = pd.read_csv('../all_data.csv')
df = df[['Client Username', 'Association Time','Map Location', 'Session Duration']]

In [5]:
df

,Client Username,Association Time,Map Location,Session Duration
0,b8d9d4b7a9b69d89f6540603ae95c7,Mon Jul 02 12:43:23 PDT 2018,MLKL LIbrary > MLKL > Eight,3 hrs 31 min 53 sec
1,9c508d14b6f397cc7003a91bea4250,Mon Jul 02 13:25:13 PDT 2018,SJSUWIFI > ENG > Floor 2,2 hrs 36 min 28 sec
2,9c508d14b6f397cc7003a91bea4250,Mon Jul 02 16:01:41 PDT 2018,SJSUWIFI > ENG > Floor 2,10 min 11 sec
3,9c508d14b6f397cc7003a91bea4250,Mon Jul 02 16:17:43 PDT 2018,SJSUWIFI > ENG > Floor 2,49 sec
4,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 11:15:04 PDT 2018,MLKL LIbrary > MLKL > Third,5 min 9 sec
5,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 11:25:19 PDT 2018,MLKL LIbrary > MLKL > Third,5 min 5 sec
6,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 11:30:26 PDT 2018,SJSUWIFI > ENG > Floor 3,5 min 6 sec
7,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 11:35:33 PDT 2018,SJSUWIFI > SSC > Floor 1,5 min 7 sec
8,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 13:14:58 PDT 2018,SJSU Outdoor > SJSU OUTDOOR,5 min 10 sec
9,d118ee8187ba90caf7e4dd620759b9,Mon Jul 02 13:20:09 PDT 2018,SJSUWIFI > ADM > Floor 1,5 min 3 sec


In [6]:
# Save data to database
#conn = sqlite3.connect('data.sqlite')
#engine = create_engine('sqlite://', echo=False)
#df.to_sql('users', con=engine)

In [7]:
#engine.execute("SELECT * FROM users").fetchall()

In [8]:
def convertMonth(month):
    if month == 'Jan': return 1
    elif month == 'Feb': return 2
    elif month == 'Mar': return 3
    elif month == 'Apr': return 4
    elif month == 'May': return 5
    elif month == 'Jun': return 6
    elif month == 'Jul': return 7
    elif month == 'Aug': return 8
    elif month == 'Sep': return 9
    elif month == 'Oct': return 10
    elif month == 'Nov': return 11
    elif month == 'Dec': return 12
    else: print('error input is not correct')
        
def convert_time_slot(time):
    temp = (time.hour * 60 + time.minute) * 60 + time.second
    return int(temp/(5*60) + 1)

def convert_time(time):
    return time.split(':')

print(convert_time('13:42:56')[0])                              

13


In [11]:
# Get number of users using an AP by date
# need modification for more fine grain time
# input is a panda dataframe
def get_number_of_users_by_date(df):
    df.shape[0]
    temp = df.iloc[0,3].split()
    #temp2 = df.iloc[0,2]
    #temp2
    result = {}
    client_list = {}
    for i in range(df.shape[0]):
        if i % 50000 == 0: print(i)
        association_time = df.iloc[i,1].split()
        temp1 = df.iloc[i,3].split()
        AP_name = df.iloc[i,2]
        client_username = df.iloc[i,0]
    
        if len(temp1) == 2: sess_duration = int(temp1[0])
        elif len(temp1) == 4: sess_duration = int(temp1[0]) * 60 + int(temp1[2])
        elif len(temp1) == 6: sess_duration = (int(temp1[0]) * 60 + int(temp1[2])) * 60 + int(temp1[4])
        else: print('error')

        d_time = datetime.datetime(year=int(association_time[5]), 
                                   month=convertMonth(association_time[1]), 
                                   day=int(association_time[2]))
        
        if AP_name not in result:
            # new AP, just add it to the list
            day_dict = {}
            day_dict[str(d_time)] = 1
            result[AP_name] = day_dict
            client_list[client_username] = []
            client_list[client_username].append(str(d_time))
        else:
            if result[AP_name].get(str(d_time)):
                # time entry already in the list
                # need to check if the user is already counted
                # if not, then add this in the list
                if client_username not in client_list:
                    result[AP_name][str(d_time)] += 1
                    client_list[client_username] = []
                    client_list[client_username].append(str(d_time))
                    #client_list[client_username] = 1
                else:
                    if str(d_time) not in client_list[client_username]:
                        result[AP_name][str(d_time)] += 1
                        client_list[client_username].append(str(d_time))
            else:
                # new time entry, just add it to the list
                result[AP_name][str(d_time)] = 1
                client_list[client_username] = []
                client_list[client_username].append(str(d_time))
                #client_list[client_username] = 1
    
    print(client_list)
    return result   
    #data_dict2[df.iloc[i,0]].append([d_time, df.iloc[i,2], sess_duration])



In [12]:
output = get_number_of_users_by_date(df)
output['Campus Housing > CV2 > 4th floor']

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000
1750000
1800000
1850000
1900000
1950000
2000000
2050000
2100000
2150000
2200000
2250000
2300000
2350000
2400000
2450000
2500000
2550000
2600000
2650000
2700000
2750000
2800000
2850000
2900000
2950000
3000000
3050000
3100000
3150000
3200000
3250000
3300000
3350000
3400000
3450000
3500000
3550000
3600000
3650000
3700000
3750000
3800000
3850000
3900000
3950000
4000000
4050000
4100000
4150000
4200000
4250000
4300000
4350000
4400000
4450000
4500000
4550000
4600000
4650000
4700000
4750000
4800000
4850000
4900000
4950000
5000000
5050000
5100000
5150000
5200000
5250000
5300000
5350000
5400000
5450000
5500000
5550000
5600000
5650000
5700000
5750000
5800000
5850000
5900000
5950000
6000000
6050000
6100000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'2018-03-17 00:00:00': 79,
 '2018-03-18 00:00:00': 89,
 '2018-03-24 00:00:00': 51,
 '2018-03-25 00:00:00': 30,
 '2018-03-10 00:00:00': 88,
 '2018-03-09 00:00:00': 79,
 '2018-04-14 00:00:00': 70,
 '2018-04-15 00:00:00': 89,
 '2018-03-11 00:00:00': 83,
 '2018-04-07 00:00:00': 70,
 '2018-04-08 00:00:00': 77,
 '2018-02-23 00:00:00': 72,
 '2018-02-24 00:00:00': 92,
 '2018-03-03 00:00:00': 70,
 '2018-03-04 00:00:00': 79,
 '2018-03-27 00:00:00': 19,
 '2018-03-28 00:00:00': 17,
 '2018-03-23 00:00:00': 50,
 '2018-03-26 00:00:00': 23,
 '2018-02-25 00:00:00': 97}

In [ ]:
# AR example
from statsmodels.tsa.ar_model import AR

# contrived dataset
data = [x + random() for x in range(1, 100)]
# fit model
model = AR(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

In [ ]:
for i in output:
    